In [418]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [419]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [420]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [421]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [422]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [423]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [424]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [425]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [426]:
_media_file_stream = open(media_pk2_path, 'rb')

In [427]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [428]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [429]:
blowfish_checksum_decoded = "Joymax Pak File"

In [430]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [431]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [432]:
Root = read_blocks_at(256)

In [433]:
blocks_in_memory = {
  "": Root
}

In [434]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [435]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [436]:

ref_shop_data = dict()
ref_shop_tab_data = dict()
ref_shop_group_data = dict()
ref_mapping_shop_group_data = []
ref_mapping_shop_with_tab_data = []

## RefShop.txt

In [437]:
ref_shop = os.path.join(server_dep, 'RefShop.txt')

In [438]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  


In [439]:
lines = get_lines(ref_shop)
for idx, line in enumerate(lines):
  values = line.split('\t')
  if len(values) != 12:
    continue
  
  Service = values[0]
  Id = values[1]
  CodeName = values[2]
  

  obj = {
    "Service": Service,
    "Id": Id,
    "CodeName": CodeName,    
  }

  ref_shop_data[CodeName] = obj

In [440]:
ref_shop_data

{'4917': {'Service': '1', 'Id': '42', 'CodeName': '4917'},
 '4904': {'Service': '1', 'Id': '42', 'CodeName': '4904'},
 '4875': {'Service': '1', 'Id': '42', 'CodeName': '4875'},
 '4906': {'Service': '1', 'Id': '42', 'CodeName': '4906'},
 '4916': {'Service': '1', 'Id': '42', 'CodeName': '4916'},
 '4981': {'Service': '1', 'Id': '42', 'CodeName': '4981'},
 '4991': {'Service': '1', 'Id': '42', 'CodeName': '4991'},
 '4962': {'Service': '1', 'Id': '42', 'CodeName': '4962'},
 '4946': {'Service': '1', 'Id': '42', 'CodeName': '4946'},
 '4930': {'Service': '1', 'Id': '42', 'CodeName': '4930'},
 '4929': {'Service': '1', 'Id': '42', 'CodeName': '4929'},
 '4919': {'Service': '1', 'Id': '42', 'CodeName': '4919'},
 '4955': {'Service': '1', 'Id': '42', 'CodeName': '4955'},
 '4940': {'Service': '1', 'Id': '42', 'CodeName': '4940'},
 '4931': {'Service': '1', 'Id': '42', 'CodeName': '4931'},
 '4928': {'Service': '1', 'Id': '42', 'CodeName': '4928'},
 '4989': {'Service': '1', 'Id': '42', 'CodeName': '4989'

## RefShopTab.txt

In [441]:
ref_shop_tab_file = os.path.join(server_dep, 'RefShopTab.txt')
ref_shop_tab_file

'server_dep\\silkroad\\textdata\\RefShopTab.txt'

In [442]:
lines = get_lines(ref_shop_tab_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      Country = values[1]
      Id = values[2]  
      CodeName = values[3]
      ref_tab_group_code_name = values[4]
      str_id_128_tab = values[5]

      ref_shop_tab_data[CodeName] = {
        "Service": Service,
        "Country": Country,
        "Id": Id,
        "CodeName": CodeName,
        "RefTabGroupCodeName": ref_tab_group_code_name,
        "StrId128Tab": str_id_128_tab
      }
    except Exception as e:
      continue

In [443]:
ref_shop_tab_data

{'LEVEL_CH_ARMOR1_TAB1': {'Service': '1',
  'Country': '42',
  'Id': '13563',
  'CodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefTabGroupCodeName': 'LEVEL_CH_ARMOR1_GROUP1',
  'StrId128Tab': 'SN_TAB_HEAVYARMOR'},
 'LEVEL_CH_ARMOR1_TAB2': {'Service': '1',
  'Country': '42',
  'Id': '13564',
  'CodeName': 'LEVEL_CH_ARMOR1_TAB2',
  'RefTabGroupCodeName': 'LEVEL_CH_ARMOR1_GROUP1',
  'StrId128Tab': 'SN_TAB_LIGHTARMOR'},
 'LEVEL_CH_ARMOR1_TAB3': {'Service': '1',
  'Country': '42',
  'Id': '13565',
  'CodeName': 'LEVEL_CH_ARMOR1_TAB3',
  'RefTabGroupCodeName': 'LEVEL_CH_ARMOR1_GROUP1',
  'StrId128Tab': 'SN_TAB_CLOTHES'},
 'LEVEL_CH_ARMOR1_TAB4': {'Service': '1',
  'Country': '42',
  'Id': '13566',
  'CodeName': 'LEVEL_CH_ARMOR1_TAB4',
  'RefTabGroupCodeName': 'LEVEL_CH_ARMOR1_GROUP2',
  'StrId128Tab': 'SN_TAB_HEAVYARMOR'},
 'LEVEL_CH_ARMOR1_TAB5': {'Service': '1',
  'Country': '42',
  'Id': '13567',
  'CodeName': 'LEVEL_CH_ARMOR1_TAB5',
  'RefTabGroupCodeName': 'LEVEL_CH_ARMOR1_GROUP2',
  'StrId128T

## RefShopGroup.txt

In [444]:
ref_shop_group_file = os.path.join(server_dep, 'RefShopGroup.txt')
ref_shop_group_file

'server_dep\\silkroad\\textdata\\RefShopGroup.txt'

In [445]:
lines = get_lines(ref_shop_group_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      Country = values[1]
      Id = values[2]  
      CodeName = values[3]
      ref_npc_code_name = values[4]

      ref_shop_group_data[CodeName] = {
        "Service": Service,
        "Country": Country,
        "Id": Id,
        "CodeName": CodeName,
        "RefNpcCodeName": ref_npc_code_name
      }
    except Exception as e:
      continue

In [446]:
ref_shop_group_data

{'GROUP_JUPITER_STORE_POTION': {'Service': '1',
  'Country': '42',
  'Id': '5642',
  'CodeName': 'GROUP_JUPITER_STORE_POTION',
  'RefNpcCodeName': 'NPC_JUPITER_SOLDIER_03'},
 'GROUP_MALL': {'Service': '1',
  'Country': '42',
  'Id': '5596',
  'CodeName': 'GROUP_MALL',
  'RefNpcCodeName': 'xxx'},
 'GROUP_MALL_ARCHEMY': {'Service': '1',
  'Country': '42',
  'Id': '5564',
  'CodeName': 'GROUP_MALL_ARCHEMY',
  'RefNpcCodeName': 'xxx'},
 'GROUP_MALL_AVATAR': {'Service': '1',
  'Country': '42',
  'Id': '5551',
  'CodeName': 'GROUP_MALL_AVATAR',
  'RefNpcCodeName': 'xxx'},
 'GROUP_MALL_CONSUME': {'Service': '1',
  'Country': '42',
  'Id': '5522',
  'CodeName': 'GROUP_MALL_CONSUME',
  'RefNpcCodeName': 'xxx'},
 'GROUP_MALL_PET': {'Service': '1',
  'Country': '42',
  'Id': '5553',
  'CodeName': 'GROUP_MALL_PET',
  'RefNpcCodeName': 'xxx'},
 'GROUP_MALL_PREMIUM': {'Service': '1',
  'Country': '42',
  'Id': '5563',
  'CodeName': 'GROUP_MALL_PREMIUM',
  'RefNpcCodeName': 'xxx'},
 'GROUP_SOTRE_CH_L

## RefMappingShopGroup.txt

In [447]:
ref_mapping_shop_group_file = os.path.join(server_dep, 'RefMappingShopGroup.txt')
ref_mapping_shop_group_file

'server_dep\\silkroad\\textdata\\RefMappingShopGroup.txt'

In [448]:
lines = get_lines(ref_mapping_shop_group_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      Country = values[1]
      Group = values[2]  
      Shop = values[3]

      obj = {
        "Service": Service,
        "Country": Country,
        "Group": Group,
        "Shop": Shop
      }

      ref_mapping_shop_group_data.append(obj)
    except Exception as e:
      continue

In [449]:
ref_mapping_shop_group_data

[{'Service': '1',
  'Country': '42',
  'Group': 'GROUP_JUPITER_STORE_POTION',
  'Shop': 'STORE_JUPITER'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_MALL',
  'Shop': 'MALL_ARCHEMY'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_MALL',
  'Shop': 'MALL_AVATAR'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_MALL',
  'Shop': 'MALL_CONSUME'},
 {'Service': '1', 'Country': '42', 'Group': 'GROUP_MALL', 'Shop': 'MALL_PET'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_MALL',
  'Shop': 'MALL_PREMIUM'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_SOTRE_CH_LEVEL2',
  'Shop': 'STORE_CH_LEVEL2'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_STORE_AR_ACCESSORY',
  'Shop': 'STORE_SD_ACCESSORY'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_STORE_AR_GUILD',
  'Shop': 'STORE_KT_GUILD'},
 {'Service': '1',
  'Country': '42',
  'Group': 'GROUP_STORE_AR_NEW_TRADE_HUNTER',
  'Shop': 'STORE_NEW_TRADE_HUNTER'},
 {'Service': '1',
  'Countr

## RefMappingShopWithTab.txt

In [450]:
ref_mapping_shop_with_tab_file = os.path.join(server_dep, 'RefMappingShopWithTab.txt')
ref_mapping_shop_with_tab_file

'server_dep\\silkroad\\textdata\\RefMappingShopWithTab.txt'

In [451]:
lines = get_lines(ref_mapping_shop_with_tab_file)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')

      Service = values[0]
      Country = values[1]
      Shop = values[2]  
      Tab = values[3]

      obj = {
        "Service": Service,
        "Country": Country,
        "Tab": Tab,
        "Shop": Shop
      }

      ref_mapping_shop_with_tab_data.append(obj)
    except Exception as e:
      continue

In [452]:
ref_mapping_shop_with_tab_data

[{'Service': '1',
  'Country': '42',
  'Tab': 'MALL_ARCHEMY_GROUP1',
  'Shop': 'MALL_ARCHEMY'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'MALL_AVATAR_GROUP1',
  'Shop': 'MALL_AVATAR'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'MALL_CONSUME_GROUP1',
  'Shop': 'MALL_CONSUME'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'MALL_PET_GROUP1',
  'Shop': 'MALL_PET'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'MALL_PREMIUM_GROUP1',
  'Shop': 'MALL_PREMIUM'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'STORE_AR_SMITH_EU_GROUP1',
  'Shop': 'STORE_AR_SMITH'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'STORE_AR_SMITH_GROUP1',
  'Shop': 'STORE_AR_SMITH'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'STORE_AR_SPECIALTY_GROUP1',
  'Shop': 'STORE_AR_SPECIALTY'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'STORE_BATTLE_ARENA_CH_GROUP1',
  'Shop': 'STORE_BATTLE_ARENA'},
 {'Service': '1',
  'Country': '42',
  'Tab': 'STORE_BATTLE_ARENA_EU_GROUP1',
  'Shop': 'STORE_BATTLE_ARENA'

## RefScrapOfPackageItem.txt

In [453]:
package_item_scrap_data = dict()

In [454]:
ref_scrap_of_package_item = os.path.join(server_dep, 'RefScrapOfPackageItem.txt')
ref_scrap_of_package_item

'server_dep\\silkroad\\textdata\\RefScrapOfPackageItem.txt'

In [455]:
ref_scrap_of_package_item_files = get_lines(ref_scrap_of_package_item)
ref_scrap_of_package_item_files

['refscrapofpackageitem_500.txt',
 'refscrapofpackageitem_1000.txt',
 'refscrapofpackageitem_1500.txt',
 'refscrapofpackageitem_2000.txt',
 'refscrapofpackageitem_2500.txt',
 'refscrapofpackageitem_3000.txt',
 'refscrapofpackageitem_3500.txt',
 'refscrapofpackageitem_4000.txt',
 'refscrapofpackageitem_4500.txt',
 'refscrapofpackageitem_5500.txt',
 '']

In [456]:
for xxx in ref_scrap_of_package_item_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) != 29:
      continue
    Service = values[0]
    Country = values[1]
    RefPackageItemCodeName = values[2]
    RefItemCodeName = values[3]
    OptLevel = values[4]
    Variance = values[5]
    Data = values[6]

    package_item_scrap_data[RefPackageItemCodeName] = {
      "Service": Service,
      "Country": Country,
      "RefPackageItemCodeName": RefPackageItemCodeName,
      "RefItemCodeName": RefItemCodeName,
      "OptLevel": OptLevel,
      "Variance": Variance,
      "Data": Data
    }


In [457]:
package_item_scrap_data

{'PACKAGE_ITEM_CART_COS_T_BEHEMOTH': {'Service': '1',
  'Country': '42',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_CART_COS_T_BEHEMOTH',
  'RefItemCodeName': 'ITEM_CART_COS_T_BEHEMOTH',
  'OptLevel': '0',
  'Variance': '0',
  'Data': '0'},
 'PACKAGE_ITEM_CART_COS_T_CAMEL1': {'Service': '1',
  'Country': '42',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_CART_COS_T_CAMEL1',
  'RefItemCodeName': 'ITEM_CART_COS_T_CAMEL1',
  'OptLevel': '0',
  'Variance': '0',
  'Data': '0'},
 'PACKAGE_ITEM_CART_COS_T_CAMEL2': {'Service': '1',
  'Country': '42',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_CART_COS_T_CAMEL2',
  'RefItemCodeName': 'ITEM_CART_COS_T_CAMEL2',
  'OptLevel': '0',
  'Variance': '0',
  'Data': '0'},
 'PACKAGE_ITEM_CART_COS_T_CAMEL3': {'Service': '1',
  'Country': '42',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_CART_COS_T_CAMEL3',
  'RefItemCodeName': 'ITEM_CART_COS_T_CAMEL3',
  'OptLevel': '0',
  'Variance': '0',
  'Data': '0'},
 'PACKAGE_ITEM_CART_COS_T_COW1': {'Service': '1',
  'Count

## RefShopGoods.txt

In [458]:
ref_shop_goods_data = []

In [459]:
ref_shop_goods = os.path.join(server_dep, 'RefShopGoods.txt')
ref_shop_goods

'server_dep\\silkroad\\textdata\\RefShopGoods.txt'

In [460]:
ref_shop_goods_files = get_lines(ref_shop_goods)
ref_shop_goods_files

['refshopgoods_50.txt',
 'refshopgoods_100.txt',
 'refshopgoods_150.txt',
 'refshopgoods_200.txt',
 'refshopgoods_250.txt',
 'refshopgoods_300.txt',
 'refshopgoods_350.txt',
 '']

In [461]:
for xxx in ref_shop_goods_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) != 13:
      continue
    Service = values[0]
    Country = values[1]
    RefTabCodeName = values[2]
    RefPackageItemCodeName = values[3]
    SlotIndex = values[4]

    obj = {
      "Service": Service,
      "Country": Country,
      "RefTabCodeName": RefTabCodeName,
      "RefPackageItemCodeName": RefPackageItemCodeName,
      "SlotIndex": SlotIndex
    }

    ref_shop_goods_data.append(obj)


In [462]:
ref_shop_goods_data

[{'Service': '1',
  'Country': '42',
  'RefTabCodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_EVENT_CH_M_HEAVY_01_AA_B_RARE',
  'SlotIndex': '4'},
 {'Service': '1',
  'Country': '42',
  'RefTabCodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_EVENT_CH_M_HEAVY_01_AA_C_RARE',
  'SlotIndex': '22'},
 {'Service': '1',
  'Country': '42',
  'RefTabCodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_EVENT_CH_M_HEAVY_01_BA_B_RARE',
  'SlotIndex': '2'},
 {'Service': '1',
  'Country': '42',
  'RefTabCodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_EVENT_CH_M_HEAVY_01_BA_C_RARE',
  'SlotIndex': '20'},
 {'Service': '1',
  'Country': '42',
  'RefTabCodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefPackageItemCodeName': 'PACKAGE_ITEM_EVENT_CH_M_HEAVY_01_FA_B_RARE',
  'SlotIndex': '5'},
 {'Service': '1',
  'Country': '42',
  'RefTabCodeName': 'LEVEL_CH_ARMOR1_TAB1',
  'RefPackageItemCodeName': 'PACKAGE_